In [5]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [6]:
%pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [28]:
from utils import convert_single_to_double_quotes
convert_single_to_double_quotes('weather_data_new.jsonl')
df_weather = pd.read_json('weather_data_new.jsonl', lines=True)
df_weather

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather,rain,wind_gust,uvi,visibility
0,1704092400,1704060451,1704101873,303.93,310.93,1008,78,299.65,22,5.29,50,"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.15},NaN,NaN,NaN
1,1704178800,1704146874,1704188304,304.48,311.48,1008,70,298.35,95,4.08,47,"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN
2,1704265200,1704233297,1704274736,304.09,310.89,1009,71,298.21,19,3.79,54,"[{'id': 801, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN
3,1704351600,1704319719,1704361168,304.40,311.40,1009,71,298.51,16,5.83,52,"[{'id': 801, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN
4,1704438000,1704406141,1704447600,303.38,310.38,1010,78,299.11,33,6.75,39,"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.18},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,1728284400,1728250695,1728293798,302.43,307.33,1008,74,297.31,36,4.21,222,"[{'id': 802, 'main': 'Clouds', 'description': ...",NaN,4.58,4.26,10000.0
281,1728370800,1728337095,1728380164,303.01,308.09,1008,71,297.18,99,4.49,215,"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN,4.98,4.32,10000.0
282,1728370800,1728337095,1728380164,303.01,308.09,1008,71,297.18,99,4.49,215,"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN,4.98,4.32,10000.0
283,1728370800,1728337095,1728380164,303.01,308.09,1008,71,297.18,99,4.49,215,"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN,4.98,4.32,10000.0


In [29]:
df_dengue = pd.read_csv('dengue_cases_22_24.csv')
df_dengue.head()

,Date,Week,Cases
0,01/01/2022,1,7
1,08/01/2022,2,1
2,15/01/2022,3,2
3,22/01/2022,4,3
4,29/01/2022,5,3


In [30]:
df_weather['dt'] = pd.to_datetime(df_weather['dt'], unit='s')
df_weather.dt = df_weather.dt.dt.date
df_weather.dt = pd.to_datetime(df_weather.dt)

In [35]:
df_weather['week'] = df_weather['dt'].dt.isocalendar().week
df_weather.head()

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather,rain,wind_gust,uvi,visibility,week
0,2024-01-01,1704060451,1704101873,303.93,310.93,1008,78,299.65,22,5.29,50,"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.15},NaN,NaN,NaN,1
1,2024-01-02,1704146874,1704188304,304.48,311.48,1008,70,298.35,95,4.08,47,"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN,1
2,2024-01-03,1704233297,1704274736,304.09,310.89,1009,71,298.21,19,3.79,54,"[{'id': 801, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN,1
3,2024-01-04,1704319719,1704361168,304.40,311.40,1009,71,298.51,16,5.83,52,"[{'id': 801, 'main': 'Clouds', 'description': ...",NaN,NaN,NaN,NaN,1
4,2024-01-05,1704406141,1704447600,303.38,310.38,1010,78,299.11,33,6.75,39,"[{'id': 500, 'main': 'Rain', 'description': 'l...",{'1h': 0.18},NaN,NaN,NaN,1


In [38]:
relevant_col = ['dt', 'week', 'temp', 'feels_like',  'pressure', 'humidity']
df_weather = df_weather[relevant_col]

In [39]:
df_weather_weekly = df_weather.groupby('week').mean()
df_weather_weekly

,dt,temp,feels_like,pressure,humidity
week,,,,,
1,2024-01-04 00:00:00,303.894286,310.865714,1009.285714,75.142857
2,2024-01-11 00:00:00,303.051429,309.288571,1009.285714,77.714286
3,2024-01-18 00:00:00,302.632857,307.614286,1009.857143,72.714286
4,2024-01-25 00:00:00,302.492857,306.968571,1013.285714,71.285714
5,2024-02-01 00:00:00,302.830000,307.444286,1012.857143,70.285714
6,2024-02-08 00:00:00,302.614286,307.707143,1012.571429,74.428571
7,2024-02-15 00:00:00,302.364286,307.007143,1012.714286,73.571429
8,2024-02-22 00:00:00,303.497143,309.060000,1011.285714,70.285714
9,2024-02-29 00:00:00,303.895714,308.310000,1011.000000,63.428571
